In [1]:
# Libs

from ISLP import load_data
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from ISLP.models import (ModelSpec as MS, summarize , poly)

# Questão 5

# a)

In [43]:
np.random.seed(1)
df = load_data('Default')

In [4]:
df_5 = df[['default', 'income', 'balance']].copy()
df_5['default'] = pd.get_dummies(df_5['default'], prefix='default', drop_first=True)


x_train = df_5[['income', 'balance']].copy()
y_train = df_5['default'].copy()

model = sm.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.173456
         Iterations 8


# b)

# i.

In [39]:
x_train, x_test, y_train, y_test = train_test_split(df_5[['income', 'balance']], df_5['default'], test_size=0.33, random_state=42)

# ii.

In [44]:
model = sm.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.170523
         Iterations 8


# iii.

In [47]:
iii_group = pd.DataFrame()

iii_group['default'] = (model.predict(x_test) > 0.5).astype(int)

# iv.

In [54]:
accuracy = accuracy_score(y_test, iii_group.default)

print(f'Nosso modelo teve uma accuracy de: {accuracy}')

Nosso modelo teve uma accuracy de: 0.9663636363636363


# c)

In [58]:
for i in range(3):
    x_train, x_test, y_train, y_test = train_test_split(df_5[['income', 'balance']], df_5['default'], test_size=0.33)

    iii_group = pd.DataFrame()
    iii_group['default'] = (model.predict(x_test) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, iii_group.default)

    print(f'Nosso modelo teve uma accuracy de: {accuracy}\n')

"""
Aparentemente os resultados sempre são bem positivos (accuracy alta, independente da amostra retirada)

Nosso modelo teve uma accuracy de: 0.9696969696969697

Nosso modelo teve uma accuracy de: 0.9687878787878788

Nosso modelo teve uma accuracy de: 0.9648484848484848



# d)

In [80]:
df['student'] = pd.get_dummies(df['student'], drop_first=True).astype(int)

df_5['student'] = df['student']

x_train, x_test, y_train, y_test = train_test_split(df_5[['income', 'balance', 'student']], df_5['default'], test_size=0.33)

model = sm.Logit(y_train, x_train).fit()

d_group = pd.DataFrame()
d_group['default'] = (model.predict(x_test > 0.5)).astype(int)

accuracy = accuracy_score(y_test, d_group.default)

print(accuracy)

"""
Continuou com valors próximos aos que já estavam

Optimization terminated successfully.
         Current function value: 0.129210
         Iterations 9
0.9718181818181818


# Questão 6

# a)

In [33]:
np.random.seed(1)
df = load_data('Default')

y_train = df['default'].copy()
y_train = pd.get_dummies(y_train, drop_first=True).astype(int)
x_train = df[['income', 'balance']].copy()

x_train = sm.add_constant(x_train)

model = sm.GLM(y_train, x_train, family=sm.families.Binomial()).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    Yes   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Tue, 03 Oct 2023   Deviance:                       1579.0
Time:                        10:32:21   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.0

# b)

In [49]:
# Função do bootstrap
def boot_fn(df, index):
    # Pegando uma sample aleatória do dataset
    data_df = df.iloc[index]

    # Modelo Logistic Regression
    y_train = data_df['default']
    x_train = data_df[['income', 'balance']]

    x_train = sm.add_constant(x_train)

    model = sm.Logit(y_train, x_train).fit(disp=0)
    
    return model.params['income'], model.params['balance']

# c)

In [50]:
df['default'] = pd.get_dummies(df['default'], drop_first=True).astype(int)

# N de repetições
repeat_times = 1000

# Arrays de valores para income e balance
income_coefs = []
balance_coefs = []

# Mágica
for _ in range(repeat_times):
    index = np.random.randint(0, len(df), len(df))
    income_coef, balance_coef = boot_fn(df, index)
    income_coefs.append(income_coef)
    balance_coefs.append(balance_coef)

In [51]:
std_income = np.std(income_coefs)
std_balance = np.std(balance_coefs)

print(f'Standard error for Income = {std_income}')
print(f'Standard error for Balance = {std_balance}')

Standard error for Income = 4.929933743996096e-06
Standard error for Balance = 0.00022260509699016862


# d)

In [ ]:
"""
Deu valores bem próximos

# Questão 7

In [61]:
df = load_data('Weekly')

y_train = pd.get_dummies(df['Direction'], drop_first=True).astype(int)
x_train = df[['Lag1', 'Lag2']].copy()
x_train = sm.add_constant(x_train)

model = sm.GLM(y_train, x_train, family=sm.families.Binomial()).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     Up   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1086
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -744.11
Date:                Tue, 03 Oct 2023   Deviance:                       1488.2
Time:                        10:55:53   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007303
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2212      0.061      3.599      0.0

In [71]:
y_train = pd.get_dummies(df['Direction'].iloc[1:], drop_first=True).astype(int)
x_train = df[['Lag1', 'Lag2']].iloc[1:].copy()
x_train = sm.add_constant(x_train)

model = sm.GLM(y_train, x_train, family=sm.families.Binomial()).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     Up   No. Observations:                 1088
Model:                            GLM   Df Residuals:                     1085
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -743.26
Date:                Tue, 03 Oct 2023   Deviance:                       1486.5
Time:                        11:03:42   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007373
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2232      0.061      3.630      0.0

# c)

In [111]:
x_new = df[['Lag1', 'Lag2']].iloc[0].copy()

# Dar uma olhada nisso
x_new = np.insert(x_new, 0, 1)

prediction_prob = model.predict(x_new)
prediction = 'Up' if prediction_prob > 0.5 else 'Down'

actual_direction = df['Direction'][0]

answer = print('Correct') if actual_direction == prediction else print('False')

False
